# Scraping avito : 

Importation des bibliotheques

In [0]:
from bs4 import BeautifulSoup 
""" Beautiful Soup transforme un document HTML complexe en un arbre complexe d’objets Python  """
import requests
""" Requests permettant d'utiliser le protocole http """
import numpy as np
import pandas as pd 

Scraper les annonces d'offres :

In [0]:
"""***************  Scraper les annonces d'offres :         ******************"""
liste_prices=[]
liste_titles=[]
liste_dates=[]
liste_types=[]
liste_categories=[]
liste_villes=[]
#scraper 250 pages d'annonces d'offres
for i in range(1,250):
    #   se connecter au site et obtenir le code source html
    url1 = "https://www.avito.ma/fr/maroc/offres?o={}".format(i)
    info1=requests.get(url1)
    # utiliser le package BeautifulSoup
    # qui "comprend" les balises html
    soup = BeautifulSoup(info1.text)     #mtnt je peux faire un recherche sur mes elements dans l'objet soup
    
    # scraper les titres 
    titles=soup.findAll('h2',attrs={ "class" : "fs14" })
    for t in titles:
      if t.text != None :
        liste_titles.append(t.text)      #ajouter le titre a la liste des titres
      else:
        liste_titles.append(np.nan)

    # scraper les prix     
    prices=soup.findAll('span',attrs={ "class" : "price_value" })
    for p in prices:
        prix=p.text
        prix=prix.replace(" ", "")     #supremer l'espace dans la chaine de caractere
        if prix!='\xa0':             # c.a.d le prix existe 
            liste_prices.append(int(prix))    # convertir le prix en entier et l'ajouter à la liste des prix
        else:                          
            prix=np.nan              # remplacer les valeurs manquante par nan   
            liste_prices.append(prix)
        
    # scraper les dates   
    dates=soup.find('div', {'class' : 'listing listing-thumbs'}).findAll("abbr","class"=="text-center")
    for d in dates:
        date = d.find("strong")
        # verifier l'existance de la date (avec != None)
        if date != None:
            # modefier la date d'aujourd'hui en format day-month
            if date.text=="Aujourd'hui":       
                date_auj=date.text.replace("Aujourd'hui","30 Mar")  
                liste_dates.append(date_auj)
            # modefier la date d'hier en format day-month    
            elif date.text=="Hier":
                date_hier=date.text.replace("Hier","29 Mar")
                liste_dates.append(date_hier)
            # les autres dates sont sous le format demander     
            else:
                liste_dates.append(date.text)
        # si le champ de la date est null on met la valeur nan        
        else:
            liste_dates.append(np.nan)
    
    # scraper le type
    types=soup.find('div', {'class' : 'listing listing-thumbs'}).findAll('span',attrs={ "class" : "item-info-extra fs14" })
    for t in types:
        t=t.find("strong")
        if t!=None:
            liste_types.append(t.text)    # ici l'annonce est professionnelle
        else:        # les autres annonces sont particuliers 
            liste_types.append("Annonce de particulier")
     
    # scraper les villes et categories
    ville_categ=soup.find('div', {'class' : 'listing listing-thumbs'}).findAll('span',attrs={ "class" : "item-info-extra fs14" })
    for vc in ville_categ:
        vc=vc.find("a")
        if vc!=None:
            l1=vc.text.split(",")                     
            liste_categories.append(l1[-2])        # extraire le categorie
            # extraire la ville
            liste_villes.append(l1[-1].replace(" ","")) 
            # replace() remplacer l'espace qui precede la chaine de caractere ville par "" (on a trouver toutes les villes commencent avec un espace ' rabat'.. 
        else:   # si le champ {ville et categorie} est vide on ajoute la valeur nan
            liste_categories.append(np.nan)
            liste_villes.append(np.nan)
    


Creer un dataframe contient les annonces d'offres

In [0]:
# representer les donnees dans un dataframe df_offres
import numpy as np
import pandas as pd
liste_offres = ["Offre" for i in range(len(liste_prices))]
ar_offres = np.array([ liste_titles , liste_prices , liste_dates , liste_types , liste_categories , liste_villes , liste_offres ]).T
rows_offres = [ "Titres" , "Prix" , "Dates" , "Types" , "Categories" , "Villes" , "Offres/Demande" ]
ind_offres = [ "article {}".format(i) for i in range(1,8716) ]

df_offres = pd.DataFrame(ar_offres,index=ind_offres,columns =rows_offres)


In [0]:
df_offres.head(20)

,Titres,Prix,Dates,Types,Categories,Villes,Offres/Demande
article 1,Recrutement massive de téléprospecteurs,5000,30 Mar,Annonce de professionnel,Offres d'emploi,Marrakech,Offre
article 2,Lecteur vitesse CD pour DDJ NUMARK CDN 88 MP3,nan,30 Mar,Annonce de professionnel,Pièces et Accessoires pour véhicules,Rabat,Offre
article 3,Téléopérateur en prise de rendez-vous,5000,30 Mar,Annonce de professionnel,Offres d'emploi,Marrakech,Offre
article 4,Créez rapidement votre entreprise,3500,30 Mar,Annonce de professionnel,Services,Casablanca,Offre
article 5,Nous recrutons en urgence 20 télé-vendeurs,5000,30 Mar,Annonce de professionnel,Offres d'emploi,Casablanca,Offre
article 6,Domicilier votre entreprise à partir de 120 d...,120,30 Mar,Annonce de professionnel,Services,Casablanca,Offre
article 7,Illi Call recrute en urgence des télé-opérate...,5000,30 Mar,Annonce de professionnel,Offres d'emploi,Casablanca,Offre
article 8,Téléopérateurs pour des sondages / enquêtes,4500,30 Mar,Annonce de professionnel,Offres d'emploi,Casablanca,Offre
article 9,Prise de rdv,4000,30 Mar,Annonce de professionnel,Offres d'emploi,Casablanca,Offre
article 10,Appartement à Fès,1900,30 Mar,Annonce de particulier,Appartements,Fès,Offre


Enregistrer nos donnees scraper dans un fichier scv 

In [0]:
# Enregistrer le dataframe dans un fichier .csv 
df_offres.to_csv ('Annonces_offres.csv', index = False, header=True)

Scraper les annonces de demandes : 

In [0]:
"""***************  Scraper les annonces de demandes :         ******************"""
""" on a que 5 pages des annonces de demandes    """
liste_prices=[]
liste_titles=[]
liste_dates=[]
liste_types=[]
liste_categories=[]
liste_villes=[]
for i in range(1,5):
    #   se connecter au site et obtenir le code source
    # le site des annonces de demandes est : https://www.avito.ma/fr/maroc/demandes?o=i i varie {1,2,3....}
    url1 = "https://www.avito.ma/fr/maroc/demandes?o={}".format(i)
    info1=requests.get(url1)
    # utiliser le package BeautifulSoup
    # qui "comprend" les balises html
    soup = BeautifulSoup(info1.text)     #mtnt je peux faire un recherche sur mes elements dans l'objet soup
    #scraper les titres 
    titles=soup.findAll('h2',attrs={ "class" : "fs14" })
    for t in titles:
        liste_titles.append(t.text)
    #scraper les prix     
    prices=soup.findAll('span',attrs={ "class" : "price_value" })
    for p in prices:
        prix=p.text
        prix=prix.replace(" ", "")     #supremer l'espace dans la chaine de caractere
        if prix!='\xa0':    
            liste_prices.append(int(prix))    # convertir le prix en entier et l'ajouter à la liste des prix
        else:                     
            prix=np.nan              # remplacer les valeurs manquante par nan   
            liste_prices.append(prix)
    #scraper les dates   
    dates=soup.find('div', {'class' : 'listing listing-thumbs'}).findAll("abbr","class"=="text-center")
    for d in dates:
        date = d.find("strong")
        #verifier l'existance de la date (avec != None)
        if date != None:
            #modefier la date d'aujourd'hui en format day-month
            if date.text=="Aujourd'hui":       
                date_auj=date.text.replace("Aujourd'hui","30 Mar")  
                liste_dates.append(date_auj)
            # modefier la date d'hier en format day-month    
            elif date.text=="Hier":
                date_hier=date.text.replace("Hier","29 Mar")
                liste_dates.append(date_hier)
            else:
                liste_dates.append(date.text)
        # si le champ de la date null on met la valeur nan        
        else:
            liste_dates.append(np.nan)
    # scraper le type
    types=soup.find('div', {'class' : 'listing listing-thumbs'}).findAll('span',attrs={ "class" : "item-info-extra fs14" })
    for t in types:
        t=t.find("strong")
        if t!=None:
            liste_types.append(t.text)
        else:
            liste_types.append("Annonce de particulier") 
    #  scraper les villes et categories
    ville_categ=soup.find('div', {'class' : 'listing listing-thumbs'}).findAll('span',attrs={ "class" : "item-info-extra fs14" })
    for vc in ville_categ:
        vc=vc.find("a")
        if vc!=None:
            l1=vc.text.split(",")               # extraire le categorie et le nom de la ville     
            liste_categories.append(l1[-2])
            liste_villes.append(l1[-1].replace(" ",""))     #trmplacer le caractere vide qui precede la ville
        else:
            liste_categories.append(np.nan)
            liste_villes.append(np.nan)

Creation du dataframe df_demandes

In [0]:
import numpy as np
import pandas as pd
liste_demandes=["Demande" for i in range(len(liste_titles))]
ar_demandes = np.array([ liste_titles , liste_prices , liste_dates , liste_types , liste_categories , liste_villes ,liste_demandes ]).T
rows_demandes = [ "Titres" , "Prix" , "Dates" , "Types" , "Categories" , "Villes" ,"Offres/Demande" ]
ind_demandes = [ "article {}".format(i) for i in range(1,141) ]

df_demandes = pd.DataFrame(ar_demandes,index=ind_demandes,columns =rows_demandes)


In [0]:
df_demandes.head(20)

,Titres,Prix,Dates,Types,Categories,Villes,Demande
article 1,أبحث عن بقعة تجارية,nan,29 Mar,Annonce de particulier,Terrains et Fermes,Safi,Demande
article 2,Cherche investisseur projet industriel rentable,nan,29 Mar,Annonce de professionnel,Business et Affaires commerciales,Casablanca,Demande
article 3,عندو شي ام الحسن,nan,29 Mar,Annonce de particulier,Animaux,Oujda,Demande
article 4,قنقلب علی ب م 320 ليصانص -1987,nan,29 Mar,Annonce de particulier,Voitures,Fès,Demande
article 5,li3ndo kijan kbar kima kano,nan,29 Mar,Annonce de particulier,Animaux,Oujda,Demande
article 6,Cherche batterie samsungnet chargeur,nan,29 Mar,Annonce de particulier,Accessoires informatique et Gadgets,Casablanca,Demande
article 7,moto -2020,nan,29 Mar,Annonce de particulier,Motos,Casablanca,Demande
article 8,Appartements pour gerence ou location LG durée,nan,29 Mar,Annonce de professionnel,Appartements,Marrakech,Demande
article 9,البحث عن شقق,nan,29 Mar,Annonce de professionnel,Appartements,Casablanca,Demande
article 10,carte mere huawie p9 light,nan,29 Mar,Annonce de particulier,Accessoires informatique et Gadgets,Meknès,Demande


Creer le fichier Annonce_demandes.csv :

In [0]:
#Enregistrer le dataframe dans un fichier Annonces_demandes.csv
df_demandes.to_csv ('Annonces_demandes.csv', index = False, header=True)